In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import torch.optim as optim

torch.set_printoptions(linewidth=120)

from torch.utils.tensorboard import SummaryWriter

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [5]:
def num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [26]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('image', grid)
tb.add_graph(network, images)
tb.close()

In [7]:
batch_size_list = [100]
lr_list = [0.01]
num_workers_list = [1]

for batch_size in batch_size_list:
    for lr in lr_list:
        for num_workers in num_workers_list:
            
            network = Network()
            loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=num_workers)
            optimiser = optim.Adam(network.parameters(), lr=lr)

            images, labels = next(iter(loader))
            grid = torchvision.utils.make_grid(images)

            tb = SummaryWriter()

            comment = f' batchsize ={batch_size} lr ={lr} num workers ={num_workers}'
            tb = SummaryWriter(comment=comment)
            tb.add_image('images', grid)
            tb.add_graph(network, images)

            for epoch in range(20):

                total_loss = 0
                total_correct = 0

                for batch in train_loader: 
                    images, labels = batch

                    preds = network(images)
                    loss = F.cross_entropy(preds, labels)

                    optimiser.zero_grad()
                    loss.backward()
                    optimiser.step()

                    total_loss += loss.item() * batch_size
                    total_correct += num_correct(preds, labels)

                tb.add_scalar('loss', total_loss, epoch)
                tb.add_scalar('number correct', total_correct, epoch)
                tb.add_scalar('accuracy', total_correct/len(train_set), epoch)

                for name, weight in network.named_parameters():
                    tb.add_histogram(name, weight, epoch)
                    tb.add_histogram(f'{name}.grad' , weight.grad, epoch)

                print('epoch:', epoch , 'loss:', total_loss, 'total correct:', total_correct)

            tb.close()


epoch: 0 loss: 35035.043278336525 total correct: 46665
epoch: 1 loss: 23251.325283944607 total correct: 51438
epoch: 2 loss: 21220.030699670315 total correct: 52203
epoch: 3 loss: 20360.329358279705 total correct: 52504
epoch: 4 loss: 19947.98551797867 total correct: 52677
epoch: 5 loss: 19555.4128870368 total correct: 52769
epoch: 6 loss: 19248.093296587467 total correct: 52948
epoch: 7 loss: 19115.586526691914 total correct: 52965
epoch: 8 loss: 18784.627535939217 total correct: 53146
epoch: 9 loss: 18405.429013073444 total correct: 53210
epoch: 10 loss: 18607.829013466835 total correct: 53185
epoch: 11 loss: 18527.47161835432 total correct: 53212
epoch: 12 loss: 18637.72555887699 total correct: 53258
epoch: 13 loss: 18020.28592079878 total correct: 53418
epoch: 14 loss: 17813.970881700516 total correct: 53488
epoch: 15 loss: 17755.11513054371 total correct: 53676
epoch: 16 loss: 17811.289358139038 total correct: 53468
epoch: 17 loss: 17207.110227644444 total correct: 53689
epoch: 18